# Apache Airflow

Installing Apache Airflow
- https://airflow.apache.org/

Vamos a instalar nuestro Apache Airflow en un Ubuntu usando multipass.


En la doc de Airflow vemos que hay dos formas: en local o con docker (asignatura de Cloud Computing) 
- Versión local: https://airflow.apache.org/docs/apache-airflow/stable/start/local.html , 

Seguimos los pasos correspondientes sobre nuestro Ubuntu:

```bash
# Airflow needs a home. `~/airflow` is the default, but you can put it
# somewhere else if you prefer (optional)
export AIRFLOW_HOME=~/airflow

# Install Airflow using the constraints file
AIRFLOW_VERSION=2.3.2
PYTHON_VERSION="$(python3 --version | cut -d " " -f 2 | cut -d "." -f 1-2)"
# For example: 3.7
CONSTRAINT_URL="https://raw.githubusercontent.com/apache/airflow/constraints-${AIRFLOW_VERSION}/constraints-${PYTHON_VERSION}.txt"
# For example: https://raw.githubusercontent.com/apache/airflow/constraints-2.3.2/constraints-3.7.txt
pip install "apache-airflow==${AIRFLOW_VERSION}" --constraint "${CONSTRAINT_URL}"

# The Standalone command will initialise the database, make a user,
# and start all components for you.
airflow standalone

# Visit localhost:8080 in the browser and use the admin account details
# shown on the terminal to login.
# Enable the example_bash_operator dag in the home page
```

### Reflexiones en este punto

- ¿Como podemos visitar la página localhost:8080 en nuestro navegador? 
- ¿Esto que significa?
- ¿Qué tipo de servicio o uso tenemos con Airflow?
- ¿Dónde está ejecutandose?


### SSH 

```bash
ssh usuario@IP

ssh ubuntu@192.168.64.8
#que ha ocurrido?

ssh -L 8080:192.168.64.8:8080 airflow@192.168.64.8

```

Averiguar la IP de nuestras máquinas con multipass
```bash
% multipass list 
Name                    State             IPv4             Image
master                  Running           192.168.64.8     Ubuntu 20.04 LTS
                                          10.1.219.64
```

Pasos necesarios para facilitar una conexión remota basada en passwords a nuestro equipo:
- En la MV de Ubuntu, cambiaremos un "no" por un "yes":  <br/>
  ```bash
  sudo nano /etc/ssh/sshd_config <br/>
  ```
  <img src="images/nanoSSHD.png"/>
  <br/>
- Reiniciamos el servicio (¿Qué es un servicio/daemon?)
  ```bash
  sudo systemctl restart sshd  
  ```

- Desde nuestra máquina Host ya podemos conectarnos:  <br/> ssh -L 8080:192.168.64.8:8080 airflow@192.168.64.8<br/> 
- Y en cualquier navegador del HOST podemos acceder a la url pertinente: *localhost:8080* <br/> 
 <img src="images/airflowcaptura.png"/>

- Finalmente, activamos el ejemplo: *example_bash_operator* . ¿Qué información nos ofrecen las diferentes pestañas?



## Creando nuestro primer DAG 

Vamos a realizar ciertos puntos del tutorial:
- https://airflow.apache.org/docs/apache-airflow/stable/tutorial.html



In [ ]:
import random as rnd
import time

def my_task1():
    # Esta función es muy compleja, que obtiene un valor después de muchisímo tiempo de ejecución
    try:
        time.sleep(5)
        value = rnd.random()
        return {"value1":value}
    except:
        none

def my_task2():
    # Esta función es muy compleja; pero no tarda tanto.
    try:
        value = rnd.random()
        time.sleep(8)
        return {"value2":value}
    except:
        none


def my_task_max(value1, value2):
    # Esta función muestra como combinar los resultados de otras tareas predecesoras
    valueMax = max(value1["value1"],value2["value2"])
    with open("/tmp/mydata.csv","a+") as f:
        f.write("%.4f,"%valueMax)

In [ ]:
from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import timedelta, datetime

default_args = {
    'depends_on_past': False,
    'email': ['airflow@example.com'],
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),

}

dag = DAG('4miPrimerDAG', 
     default_args=default_args,
     start_date = datetime(2019,1,1),#datetime.now()-timedelta(minutes=1),
     schedule_interval = timedelta(minutes=2) 
    )

In [ ]:
t1 = PythonOperator(dag=dag,
        task_id='my_task1',
        python_callable=my_task1)

t2 = PythonOperator(dag=dag,
        task_id='my_task2',
        python_callable=my_task2)


t3 = PythonOperator(dag=dag,
        task_id='my_MAX',
        op_kwargs={'value1': t1.output, 'value2': t2.output},
        python_callable=my_task_max)

In [ ]:
# [t1, t2] >> t3

t1.set_downstream(t3)
t2.set_downstream(t3)

In [ ]:
# Todo junto

from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import timedelta, datetime

import random as rnd
import time

def my_task1():
    # Esta función es muy compleja, que obtiene un valor después de muchisímo tiempo de ejecución
    try:
        time.sleep(5)
        value = rnd.random()
        return {"value1":value}
    except:
        none

def my_task2():
    # Esta función es muy compleja; pero no tarda tanto.
    try:
        value = rnd.random()
        time.sleep(8)
        return {"value2":value}
    except:
        none


def my_task_max(value1, value2):
    # Esta función muestra como combinar los resultados de otras tareas predecesoras
    valueMax = max(value1["value1"],value2["value2"])
    with open("/tmp/mydata.csv","a+") as f:
        f.write("%.4f,"%valueMax)

default_args = {
    'depends_on_past': False,
    'email': ['airflow@example.com'],
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),

}

dag = DAG('4miPrimerDAG', 
     default_args=default_args,
     start_date = datetime(2019,1,1),#datetime.now()-timedelta(minutes=1),
     schedule_interval = timedelta(minutes=2) 
    )

t1 = PythonOperator(dag=dag,
        task_id='my_task1',
        python_callable=my_task1)

t2 = PythonOperator(dag=dag,
        task_id='my_task2',
        python_callable=my_task2)


t3 = PythonOperator(dag=dag,
        task_id='my_MAX',
        op_kwargs={'value1': t1.output, 'value2': t2.output},
        python_callable=my_task_max)


# [t1, t2] >> t3

t1.set_downstream(t3)
t2.set_downstream(t3)

# Recordatorio de algunos comandos
# python3 mydag.py
# airflow db init

# airflow tasks list miPrimerDAG
# airflow tasks test miPrimerDAG my_task1
# airflow dags trigger miPrimerDAG


## References
- Data Pipelines with Apache Airflow. Manning Publications Bas P. Harenslak, Julian Rutger de Ruiter. 2021
- https://betterdatascience.com/apache-airflow-run-tasks-in-parallel/
